In [ ]:
"""
セグメント木


配列にたいして、
・区間[a,b)について最小やら最大やらを求める
・要素iの値を変更する
を高速に行える

詳しくは：https://www.slideshare.net/iwiwi/ss-3578491?ref=https://juppy.hatenablog.com/entry/2019/05/02/%E8%9F%BB%E6%9C%AC_python_%E3%82%BB%E3%82%B0%E3%83%A1%E3%83%B3%E3%83%88%E6%9C%A8_%E7%AB%B6%E6%8A%80%E3%83%97%E3%83%AD%E3%82%B0%E3%83%A9%E3%83%9F%E3%83%B3%E3%82%B0_Atcoder

"""

class SegmentTree:
    # セグメント木として初期化する配列、単位元、[a,b)に実行する関数
    def __init__(self, lst, ide_ele, segfunc):
        self.ide_ele = ide_ele
        self.func = segfunc
        # lstのサイズより大きい2^xで最小の値
        self.size = 2**(len(lst) - 1).bit_length()
        self.seg_lst = [self.ide_ele] * 2 * self.size

        """
        値を詰めていく
        ノードiの親：(i-1) // 2
        ノードiの子：2*i+1 と　2*i+2

        """
        for i in range(len(lst)):
            self.seg_lst[i + self.size - 1] = lst[i]

        # クエリが実行されたときに取り出せるように処理をしておく
        # この辺は何がしたいかによって自分で関数用意する
        for i in range(self.size-2, -1, -1):
            # 子から親に向かって結果を吸い上げるような形
            self.seg_lst[i] = self.func(self.seg_lst[2*i + 1], self.seg_lst[2*i + 2])

    def update(self, i, num):
        # 一番下の子の部分を更新するため、配列中のindexからseg_lstのindexに変える
        i += self.size - 1
        self.seg_lst[i] = num
        while i:
            # 親に更新を伝播
            i = (i - 1) // 2
            self.seg_lst[i] = self.func(self.seg_lst[2*i + 1], self.seg_lst[2*i + 2])

    def query(self, l, r):
        # 範囲がおかしい場合
        if r <= l:
            return self.ide_ele

        # seg_lst内でのindexに直す
        l += self.size - 1
        # 
        r += self.size - 2
        ret = self.ide_ele
        # 区間が閉じ切る = 同じ親のところに戻るまで
        while r - l > 1:
            # lの下位1ビットが立っていない。たぶんだが、あるノードの左右の子の左が範囲に入ってるかどうかの判断で、入ってないときか？
            if l & 1 == 0:
                # 一番初めは単位減であるretとlについてfuncをかけることになるので、やはり左の子が範囲に入ってないとき！
                ret = self.func(ret, self.seg_lst[l])
            # こっちは右の子が入ってるかどうかで、入っている場合の処理だと思う
            if r & 1 == 1:
                # 一番最初は単位元とrの処理なので、右側が範囲に入ってない時。
                ret = self.func(ret, self.seg_lst[r])
                r -= 1

            # それぞれの親に移動というか、一個上のdepthに移動
            l = l // 2
            r = (r - 1) // 2

        # retには下から積み上げてきたself.funcの結果が入ってるので、たどり着いた親の所で同じく処理を実行して戻す
        if l == r:
            ret = self.func(ret, self.seg_lst[l])
        else:
            ret = self.func(self.func(ret, self.seg_lst[l]), self.seg_lst[r])
        
        return ret

In [ ]:
"""
AtCoder Beginner Contest 125  C - GCD on Blackboard

配列中の数字を一つ好きな値に書き換えて最大公約数を計算するときの最大値を求める
・書き換える→取り除くと考えてよい。
・全体のGCD = GCD(前半のGCD, 後半のGCD) のように、分割したり順番を入れ替えたりしても結果は変わらない。
なので、左からの累積GCDと、右からの累積GCDをあらかじめとっておくことで、要素を一個除いたときの全体のGCDも求められる。　

区間に対する処理 -> セグメント木いけるやん
"""
class SegmentTree:
    # セグメント木として初期化する配列、単位元、[a,b)に実行する関数
    def __init__(self, lst, ide_ele, segfunc):
        self.ide_ele = ide_ele
        self.func = segfunc
        # lstのサイズより大きい2^xで最小の値
        self.size = 2**(len(lst) - 1).bit_length()
        self.seg_lst = [self.ide_ele] * 2 * self.size

        """
        値を詰めていく
        ノードiの親：(i-1) // 2
        ノードiの子：2*i+1 と　2*i+2

        """
        for i in range(len(lst)):
            self.seg_lst[i + self.size - 1] = lst[i]

        # クエリが実行されたときに取り出せるように処理をしておく
        # この辺は何がしたいかによって自分で関数用意する
        for i in range(self.size-2, -1, -1):
            # 子から親に向かって結果を吸い上げるような形
            self.seg_lst[i] = self.func(self.seg_lst[2*i + 1], self.seg_lst[2*i + 2])

    def update(self, i, num):
        # 一番下の子の部分を更新するため、配列中のindexからseg_lstのindexに変える
        i += self.size - 1
        self.seg_lst[i] = num
        while i:
            # 親に更新を伝播
            i = (i - 1) // 2
            self.seg_lst[i] = self.func(self.seg_lst[2*i + 1], self.seg_lst[2*i + 2])

    def query(self, l, r):
        # 範囲がおかしい場合
        if r <= l:
            return self.ide_ele

        # seg_lst内でのindexに直す
        l += self.size - 1
        # 
        r += self.size - 2
        ret = self.ide_ele
        # 区間が閉じ切る = 同じ親のところに戻るまで
        while r - l > 1:
            # lの下位1ビットが立っていない。たぶんだが、あるノードの左右の子の左が範囲に入ってるかどうかの判断で、入ってないときか？
            if l & 1 == 0:
                # 一番初めは単位減であるretとlについてfuncをかけることになるので、やはり左の子が範囲に入ってないとき！
                ret = self.func(ret, self.seg_lst[l])
            # こっちは右の子が入ってるかどうかで、入っている場合の処理だと思う
            if r & 1 == 1:
                # 一番最初は単位元とrの処理なので、右側が範囲に入ってない時。
                ret = self.func(ret, self.seg_lst[r])
                r -= 1

            # それぞれの親に移動というか、一個上のdepthに移動
            l = l // 2
            r = (r - 1) // 2

        # retには下から積み上げてきたself.funcの結果が入ってるので、たどり着いた親の所で同じく処理を実行して戻す
        if l == r:
            ret = self.func(ret, self.seg_lst[l])
        else:
            ret = self.func(self.func(ret, self.seg_lst[l]), self.seg_lst[r])
        
        return ret

    
from fractions import gcd
N = int(input())
A = list(map(int, input().split()))

seg_tree = SegmentTree(A, 0, gcd)

ans = -1

for i in range(N):
    # 要素iを除いたときの全体のGCD
    ans = max(ans, gcd(seg_tree.query(0, i), seg_tree.query(i+1, N)))

print(ans)

In [ ]:
"""


"""